In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn import neighbors
%matplotlib inline

In [ ]:
train = pd.read_csv("TrainMatrix.csv",index_col = 0)
print(train.shape)
test = pd.read_csv("TestValues.csv")
print(test.shape)

In [ ]:
s = train.sample(5)
mans = s.index
supps = s.columns[0:5]
train[train.index.isin(mans)]

In [ ]:
tmans = test.sample(5).index
test[test.index.isin(tmans)]

In [ ]:
## Find mean of each users ratings
user_ratings_mean = np.mean(train, axis = 1)
## From each row (User rating) subtract the mean of that user's ratings
## Thus, items that have not been rated have a value of 0
train_modified = train.apply(lambda x: x-x.mean(), axis = 1).fillna(0)

train_modified[train_modified.index.isin(mans)]

In [ ]:
## Define the number of nearest neighbors that we want as our default
## This number will change later in a certain implementation
## But because we use the number 100 later, we're keeping it like this.
k = 100

## Initialize our nearest neighbors classes
## Find the nearest based on the standardized matrix

## Nearest neighbors using euclidean distance
nnEuc = neighbors.NearestNeighbors(n_neighbors = k,
                                   metric = 'euclidean').fit(train_modified)
## Nearest neighbors using cosine distance
nnCos = neighbors.NearestNeighbors(n_neighbors = k,
                                   metric = 'cosine').fit(train_modified)

In [ ]:
## Transform the training matrix into a slightly more usable format for a future purpose
knownRatings = train.stack().reset_index()
knownRatings.columns = ["Manufacturer_ID","Supplier_ID","Rating"]
## Find the ratings that we don't know
## We're only using the test dataframe because 
## If we tried to find the value for ALL the datapoints/combinations
## It would take far too long
unknownRatings = test[['Manufacturer_ID',"Supplier_ID","Rating"]]
knownRatings[knownRatings['Manufacturer_ID'].isin(mans)].sample(5)

In [ ]:
unknownRatings.sample(5)

In [ ]:
def getPredictedRatingEuc(row):
    
    ## Number of neighbors we care about
    k = 10
    ## Name of manufacturer we care about
    manu = row['Manufacturer_ID']
    ## Name of supplier we care about
    supp = row['Supplier_ID']
    
    ## Find all the ratings given to that supplier
    ratingsdf = knownRatings[knownRatings['Supplier_ID'] == supp]
    lrd = len(ratingsdf)
    ## We're just brute force choosing 100 neighbors. We will trim this down later.
    ## There probably is a more dynamic way to do this other than choosing 100 neighbors and trimming it down.
    ## But for our purposes, this is fine.
    ## For slight efficiency purposes we've added a comparison to the length of the ratings dataframe.
    ## This probably won't help too much, but just in case.
    dists, neighbors = nnEuc.kneighbors([train_modified.loc[manu]], n_neighbors = min(100,lrd))
    dists = dists[0]
    
    ## Neighbors just has the index right now. We want the Manufacturer_ID
    neighbors = np.array(train_modified.index[neighbors][0])
    
    ## Trim the neighbors to only the closest ones that have rated what we're looking for
    hasRated = np.isin(neighbors,
                       ratingsdf['Manufacturer_ID'])
        
    ## Distances of closest neighbors
    dists = dists[hasRated]
    
    ## Names of closest neighbors
    neighbors = neighbors[hasRated]
    neighbors = neighbors[dists>0]
    neighbors = neighbors[:k]
    
    dists = dists[dists>0]
    dists = dists[:k]
    
    ## We want bigger Euclidean distances to have less weight
    ## And smaller distances to have more weight
    exp = lambda x : (x)**-1
    vfunc = np.vectorize(exp)
    dists = vfunc(dists)
    
    ## The ratings the closest neighbors have given to your supplier
    ratingsdf = ratingsdf[ratingsdf['Manufacturer_ID'].isin(neighbors)]
    
    ## Define a lambda function to get the rating that the neighbor has given the item
    ## This helps speed up the process
    
    ## 0 comes from the first row value
    ## 2 comes from the index of the "Rating" column
    ratingFinder = lambda x : ratingsdf[ratingsdf['Manufacturer_ID']==x].iat[0,2]
    
    ## Vectorize this function so we can quickly apply it
    vfunc=np.vectorize(ratingFinder)
    
    ## Get a vector of the ratings given
    ratingsVector = vfunc(neighbors)
    
    ## This is our estimated rating using the weighted average method we spoke about
    estimatedRating = np.divide(np.sum(np.multiply(dists,
                                                   ratingsVector)),
                                np.sum(dists))
    
    return estimatedRating

In [ ]:
def getPredictedRatingCos(row):
    
    ## Number of neighbors we care about
    k = 10
    ## Name of manufacturer we care about
    manu = row['Manufacturer_ID']
    ## Name of supplier we care about
    supp = row['Supplier_ID']
    
    ## Find all the ratings given to that supplier
    ratingsdf = knownRatings[knownRatings['Supplier_ID'] == supp]
    lrd = len(ratingsdf)
    ## We're just brute force choosing 100 neighbors. We will trim this down later.
    ## There probably is a more dynamic way to do this other than choosing 100 neighbors and trimming it down.
    ## But for our purposes, this is fine.
    ## For slight efficiency purposes we've added a comparison to the length of the ratings dataframe.
    ## This probably won't help too much, but just in case.
    dists, neighbors = nnCos.kneighbors([train_modified.loc[manu]], n_neighbors = min(100,lrd))
    dists = dists[0]
    
    ## Neighbors just has the index right now. We want the Manufacturer_ID
    neighbors = np.array(train_modified.index[neighbors][0])
    
    ## Trim the neighbors to only the closest ones that have rated what we're looking for
    hasRated = np.isin(neighbors,
                       ratingsdf['Manufacturer_ID'])
        
    ## Distances of closest neighbors
    dists = dists[hasRated]
    dists = dists[:k]
    ## Taking into account negative cosines
    dists = dists+1
    
    ## Names of closest neighbors
    neighbors = neighbors[hasRated]
    neighbors = neighbors[:k]
    
    ## The ratings the closest neighbors have given to your supplier
    ratingsdf = ratingsdf[ratingsdf['Manufacturer_ID'].isin(neighbors)]
    
    ## Define a lambda function to get the rating that the neighbor has given the item
    ## This helps speed up the process
    
    ## 0 comes from the first row value
    ## 2 comes from the index of the "Rating" column
    ratingFinder = lambda x : ratingsdf[ratingsdf['Manufacturer_ID']==x].iat[0,2]
    
    ## Vectorize this function so we can quickly apply it
    vfunc=np.vectorize(ratingFinder)
    
    ## Get a vector of the ratings given
    ratingsVector = vfunc(neighbors)
    
    ## This is our estimated rating using the weighted average method we spoke about
    estimatedRating = np.divide(np.sum(np.multiply(dists,
                                                   ratingsVector)),
                                np.sum(dists))
    
    return estimatedRating

In [ ]:
## Utility function we define 
## to print out the sum, mean, and standard deviation
## Of a column
def printstats(df,col):
    print(col)
    print("SUM: ",df[col].sum(), 
          "\nMEAN: ",df[col].mean(), 
          "\nSTD: ",df[col].std())
    print()

In [ ]:
## Take a sample from our testing dataframe
## We take a sample because we don't have the processing power
## To test all of the test data simultaneously
## But because we have a VERY LARGE sample size
## We can use this data to extrapolate
tempdf = unknownRatings.sample(1000)

## Find the predicted rating using Euclidean distance
tempdf['EucRating'] = tempdf.apply(getPredictedRatingEuc,axis=1)
## Find the predicted rating using Cosine distance
tempdf['CosRating'] = tempdf.apply(getPredictedRatingCos,axis=1)
## Predict the mean rating of this supplier
## In other words, given the training data we have
## What is a average rating given to this supplier
## We use this to compare performances of other methodologies
tempdf['ItemMeanRating'] = tempdf['Supplier_ID'].apply(lambda x : train[x].mean())
tempdf['UserMeanRating'] = tempdf['Manufacturer_ID'].apply(lambda x: train.mean(axis=1)[x])

## To show what the data looks like right now
tempdf.sample(10)

In [ ]:
## With this code, we're finding the squared difference of each rating methodology
## From the actual rating given
tempdf['EucDiff'] = (tempdf['Rating']-tempdf['EucRating'])**2
tempdf['MeanDiff'] = (tempdf['Rating']-tempdf['ItemMeanRating'])**2
tempdf['CosDiff'] = (tempdf['Rating']-tempdf['CosRating'])**2
tempdf['UserDiff'] = (tempdf['Rating']-tempdf['UserMeanRating'])**2

## Print out the statistics for each of the errors of each methodology
printstats(tempdf,'EucDiff')
printstats(tempdf,'CosDiff')
printstats(tempdf,'MeanDiff')
printstats(tempdf,'UserDiff')

In [ ]:
foo = tempdf.sample(10)
foo

In [ ]:
np.sqrt(foo.mean())

### Let's test if on average, using Cosine Collaborative Filtering returns less error than just predicting the Mean Value

In [ ]:
diffs = []
ediffs = []
cdiffs = []
av = []
co = []
eu = []
ur = []
udiffs = []

for i in range(100):
    if i%10 == 0 or i>95:
        print(i)
    ## Take a sample from our testing dataframe
    ## We take a sample because we don't have the processing power
    ## To test all of the test data simultaneously
    ## But because we have a large sample size
    ## We can use this data to extrapolate
    tempdf = unknownRatings.sample(100)

    ## Find the predicted rating using Cosine distance
    tempdf['CosRating'] = tempdf.apply(getPredictedRatingCos,axis=1)

    ## Find the predicted rating using Cosine distance
    tempdf['EucRating'] = tempdf.apply(getPredictedRatingEuc,axis=1)

    ## Predict the mean rating of this supplier
    ## In other words, given the training data we have
    ## What is a average rating given to this supplier
    tempdf['MeanRating'] = tempdf['Supplier_ID'].apply(lambda x : train[x].mean())
    tempdf['UserRating'] = tempdf['Manufacturer_ID'].apply(lambda x : train.loc[x].mean())

    tempdf['MeanDiff'] = (tempdf['Rating']-tempdf['MeanRating'])**2
    tempdf['CosDiff'] = (tempdf['Rating']-tempdf['CosRating'])**2
    tempdf['EucDiff'] = (tempdf['Rating']-tempdf['EucRating'])**2
    tempdf['UserDiff'] = (tempdf['Rating']-tempdf['UserRating'])**2

    ## Append the difference in RMSE to the array
    ## If this number is positive, this means that the MSE using mean to predict
    ## Is greater than the MSE of using cosine to predict
    ## For this specific instance
    
    md = np.sqrt(tempdf['MeanDiff'].mean())
    cd = np.sqrt(tempdf['CosDiff'].mean())
    ed = np.sqrt(tempdf['EucDiff'].mean())
    ud = np.sqrt(tempdf['UserDiff'].mean())
    
    av.append(md)
    co.append(cd)
    eu.append(ed)
    ur.append(ud)
    diffs.append(md-cd)
    ediffs.append(md-ed)
    cdiffs.append(cd-ed)

In [ ]:
demo = test[test['Manufacturer_ID']=='M_2299517']
demo['Predicted_Rating']=demo.apply(getPredictedRatingCos,axis=1)
demo.sort_values('Predicted_Rating', ascending = False)[['Supplier_ID',"Predicted_Rating"]]

In [ ]:
alphaLevel = 0.05

In [ ]:
from scipy import stats
## One way anova to test if the means of all three methods are the same
F, p = stats.f_oneway(av, co, eu, ur)
## Because P < 0.05 
## We can say there's enough evidence to conclude 
## That the means of all these methods are not the same
## Which means that the average RMSE of each of these methods differs
"P = ", p, "IS P LESS THAN ALPHA?:", p<alphaLevel

## Results of above test

We've proven that the average RMSE of the three methodologies is not the same (i.e. at least one differs from the other two). Now we need to find which ones differ.

In [ ]:
## Ttest for equality of means between cosine and average item rating
stats.ttest_ind(a=av, b=co).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=av, b=co).pvalue, "t statistic",stats.ttest_ind(a=av, b=co).statistic

In [ ]:
## Ttest for equality of means between average item rating and euclidean
stats.ttest_ind(a=av, b=eu).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=av, b=eu).pvalue, "t statistic",stats.ttest_ind(a=av, b=eu).statistic

In [ ]:
## Ttest for equality of means between euclidean and cosine
stats.ttest_ind(a=co, b=eu).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=co, b=eu).pvalue, "t statistic",stats.ttest_ind(a=co, b=eu).statistic

In [ ]:
## Ttest for equality of means between user mean and average item rating
stats.ttest_ind(a=ur, b = av).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=ur, b=av).pvalue, "t statistic",stats.ttest_ind(a=ur, b=av).statistic

In [ ]:
## Ttest for equality of means between user mean and cosine
stats.ttest_ind(a=ur, b = co).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=ur, b=co).pvalue, "t statistic",stats.ttest_ind(a=ur, b=co).statistic

In [ ]:
## Ttest for equality of means between user mean and euclidean
stats.ttest_ind(a=ur, b = eu).pvalue < alphaLevel

In [ ]:
"P Value",stats.ttest_ind(a=ur, b=eu).pvalue, "t statistic",stats.ttest_ind(a=ur, b=eu).statistic

## So now we've proven that none of the average RMSEs of the methods are equal to each other. 

Now we can use basic statistics to find which method is the best. We do this by looking at the means of each method. The one with the lowest mean is the best method.

In [ ]:
plt.errorbar(x = ["Euclidean",
                  "Mean Item Rating",
                  "Cosine",
                  "Mean User Rating"],
             y = [np.mean(eu),
                  np.mean(av),
                  np.mean(co),
                  np.mean(ur)],
             linestyle = 'None',
             yerr = [np.std(x)/np.sqrt(len(x)) for x in [eu,av,co,ur]],
             marker='.',
             capsize=3)

plt.annotate(np.round(np.mean(co),3), (0.1,np.mean(co)))

plt.annotate(np.round(np.mean(eu),3), (1.1,np.mean(eu)))

plt.annotate(np.round(np.mean(av),3), (2.1,np.mean(av)))

plt.annotate(np.round(np.mean(ur),3), (2.65,np.mean(ur)))

plt.title("Average RMSE")
plt.xlabel("METHODS")
plt.ylabel("RMSE")